
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Restart AWS EC2 Instance using unSkript legos.</b>
</div>

<br>

<center><h2>Restart EC2 Instance</h2></center>

# Steps Overview 
    1 Get instance_ids from action Filter AWS EC2 Instances by tag(Inputs: tag_key,tag_value,region)
    2 Pass the array of instance_id:list and region:string to the Restart AWS EC2 instance action
    3 Get the instance details by passing the instance_ids:string and region:string

Here we will use unSkript Filter AWS EC2 Instances by tag Lego. This lego takes tag_key, tag_value, region as input. This inputs is used to findout all the instances available on specified region.

In [3]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype
@beartype
def aws_filter_ec2_by_tags_printer(output):
    if output is None:
        return
    pprint.pprint({"Instances": output})


@beartype
def aws_filter_ec2_by_tags(handle, tag_key: str, tag_value: str, region: str) -> List:
    """aws_filter_ec2_by_tags Returns an array of instances matching tags.

        :type nbParamsObj: object
        :param nbParamsObj: Object containing global params for the notebook.

        :type credentialsDict: dict
        :param credentialsDict: Dictionary of credentials info.

        :type inputParamsJson: string
        :param inputParamsJson: Json string of the input params.

        :rtype: Array of instances matching tags.
    """
    # Input param validation.

    ec2Client = handle.client('ec2', region_name=region)
    res = aws_get_paginator(ec2Client, "describe_instances", "Reservations",
                            Filters=[{'Name': 'tag:' + tag_key, 'Values': [tag_value]}])

    result = []
    for reservation in res:
        for instance in reservation['Instances']:
            result.append(instance['InstanceId'])
    return result


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "region": "region",
    "tag_key": "tag_key",
    "tag_value": "tag_value"
    }''')
task.configure(outputName="InstanceList")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_ec2_by_tags, lego_printer=aws_filter_ec2_by_tags_printer, hdl=hdl, args=args)

Here we will use unSkript Restart AWS EC2 Instances by tag Lego. This lego takes instance_ids and region as input. This inputs is used to restart the instances which we get using above Filter AWS EC2 Instances by tag Lego. We pass the instances IDs list to the lego.

In [4]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_restart_ec2_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_restart_ec2_instances(handle, instance_ids: List, region: str) -> Dict:
    """aws_restart_instances Restarts instances.

        :type nbParamsObj: object
        :param nbParamsObj: Object containing global params for the notebook.

        :type credentialsDict: dict
        :param credentialsDict: Dictionary of credentials info.

        :type inputParamsJson: string
        :param inputParamsJson: Json string of the input params.

        :rtype: Dict with the stopped instances state info.
    """

    ec2Client = handle.client('ec2', region_name=region)
    res = ec2Client.reboot_instances(InstanceIds=instance_ids)
    return res


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "instance_ids": "InstanceList",
    "region": "region"
    }''')
task.configure(outputName="restartmsg")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_restart_ec2_instances, lego_printer=aws_restart_ec2_instances_printer, hdl=hdl, args=args)

Here we will use unSkript Get AWS Instance Details Lego to get the details of the instances. This lego takes instance_ids and region as input. This input is used to get details of instances which we received in the first Action of this Runbook.

In [5]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import pprint
from beartype import beartype

def aws_get_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_get_instance_details(
    handle,
    instance_id: str,
    region: str,
) -> Dict:

    ec2client = handle.client('ec2', region_name=region)
    instances = []
    response = ec2client.describe_instances(
        Filters=[{"Name": "instance-id", "Values": [instance_id]}])
    for reservation in response["Reservations"]:
        for instance in reservation["Instances"]:
            instances.append(instance)

    return instances[0]



task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "instance_id": "iter_item",
    "region": "region"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "InstanceList",
    "iter_parameter": "instance_id"
    }''')
task.configure(outputName="InstanceDetails")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_instance_details, lego_printer=aws_get_instances_printer, hdl=hdl, args=args)

In [6]:
import pprint
from pydantic import BaseModel, Field
from typing import Dict
from tabulate import tabulate

output = InstanceDetails
instance_list = InstanceList
def Instance_Details(output,instance_list: list):
    data1 = []
    Header = ""
    for instance_id in instance_list:
        if instance_id in output.keys():
            output1 = output[instance_id]
            if isinstance(output1, (list, tuple)):
                for item in output1:
                    print(f'item: {item}')
            elif isinstance(output1, dict):
                for key,value in output1.items():
                    if isinstance(value, (list)):
                        pass
                    else:
                        if key == "InstanceId":
                            Header = value
                        data1.append([key,value])
                print(f'\n\033[1m Table for Instance ID : {Header} \033[0;0m')
                print(tabulate(data1))
            else:
                print(f'Output for {task.name}')
                print(output1)
                
Instance_Details(output,instance_list)

In this Runbook, we demonstrated the use of unSkript's AWS legos to restart the AWS EC2 instances and get the details. To view the full platform capabilities of unSkript please visit https://unskript.com